# This is to generate a random combination of serveral parameters within certain range

In [8]:
import csv, random, os
import pandas as pd
import numpy as np

In [5]:
# ONLY run once! 
# first, we need to define the bounds for all 8 parameters
bounds_Dint = [3e-20,4e-19]
bounds_Li_decay = [4e-7,2.5e-6 ]
bounds_k_LiP = [1e-10, 1e-8]
bounds_LAM_Nep = [1e-9, 3e-7]
bounds_k_Necr = [1e-21, 9e-20]
bounds_elely_res = [0.999, 1.4]
bounds_V_EC  =  [4.8e-5, 6.67e-5]
bounds_R_cont     = [0.0105, 0.02 ]
Bounds = [
    bounds_Dint,   bounds_Li_decay, bounds_k_LiP,
    bounds_LAM_Nep,bounds_k_Necr,   bounds_elely_res,
    bounds_V_EC,bounds_R_cont]
parameter_names = [
    'Inner SEI lithium interstitial diffusivity [m2.s-1]', 
    'Dead lithium decay constant [s-1]', 
    'Lithium plating kinetic rate constant [m.s-1]', 
    'Negative electrode LAM constant proportional term [s-1]', 
    'Negative electrode cracking rate', 
    'Initial electrolyte excessive amount ratio', 
    'EC partial molar volume [m3.mol-1]', 
    'Contact resistance [Ohm]'
]
def generate_combinations(Bounds, Num_tot):
    lower_bounds = []
    upper_bounds = []
    for bound in Bounds:
        lower_bounds.append(bound[0])
        upper_bounds.append(bound[1])
    combinations = []
    for _ in range(Num_tot):
        combination = []
        for lower, upper in zip(lower_bounds, upper_bounds):
            value = random.uniform(lower, upper)
            combination.append(value)
        combinations.append(combination)
    return combinations

def save_combinations_to_csv(combinations, parameter_names, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(parameter_names)  # Write parameter names as the first row
        for combination in combinations:
            writer.writerow(combination)

# Usage example:
BasicPath =  os.path.expanduser("~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
Target  = f'/Random_1000sets/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)
Random_1000 = generate_combinations(Bounds, Num_tot=1000)
filename = BasicPath+Target+'Random_1000sets.csv'
save_combinations_to_csv(Random_1000, parameter_names, filename)
print("Combinations saved to 'Random_1000sets.csv' file.")


Combinations saved to 'Random_1000sets.csv' file.


In [ ]:
# reload
def load_combinations_from_csv(filename):
    dataframe = pd.read_csv(filename)
    parameter_names = dataframe.columns.tolist()
    combinations = dataframe.values.tolist()
    return parameter_names, combinations

# Usage example:
filename = BasicPath+Target+'Random_1000sets.csv'
parameter_names, combinations = load_combinations_from_csv(filename)


In [15]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize
import matplotlib.pyplot as plt;
import imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl; 
fs=17; 
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [23]:
## Global setting: change with PC or HPC
# Path_NiallDMA = "InputData/" # for HPC

# Path_NiallDMA = "D:/OneDrive - Imperial College London/SimDataSave/InputData/" # for Windows
Path_NiallDMA = os.path.expanduser("~/EnvPBGEM_Linux/SimSave/InputData/") # for Linux
# BasicPath=os.getcwd() # for HPC
# path for windows's pybamm in office PC 
# BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P2_R9_Dim/Test'  
# for linux pybamm in office pc:
BasicPath =  os.path.expanduser("~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip/Rio_Code/Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2 import * # directly do this for HPC

# Set Experiment : Exp-2
index_exp = 2 # index for experiment set, can now go for 2,3,5
Round_No = "debug_shape"  # index to identify different rounds of running 
pool_no = 8
Runshort = True # True

# specify case 1-10
Scan_start = 1;    Scan_end = 10; 
Indexs =np.arange(Scan_start-1,Scan_end)

if Runshort == False:
    if index_exp == 2:
        tot_cyc = 6192; cyc_age = 516; # should be 6192 but now run shorter to be faster
    if index_exp == 3:
        tot_cyc = 6180; cyc_age = 515;
    if index_exp == 5:
        tot_cyc = 1170; cyc_age = 78;
else:
    if index_exp == 2:
        tot_cyc = 6; cyc_age = 1;
    if index_exp == 3:
        tot_cyc = 8; cyc_age = 2;
    if index_exp == 5:
        tot_cyc = 8; cyc_age = 2;

para_dict_Same = {
   "Ageing temperature":25, # 25, 40 
   "Total ageing cycles":int(tot_cyc), # 
   "Ageing cycles between RPT":int(cyc_age), # 
   "Update cycles for ageing": int(cyc_age),
   "Cycles within RPT":1,
   "RPT temperature":25,
   "Mesh list":[5,5,5,60,20],   # Simon uses 30
   "Para_Set": "OKane2023", # Li2023_Coupled
   "Model option":{
        "contact resistance": "true",
        "open-circuit potential": "current sigmoid",
        "thermal": "lumped",
        "SEI": "interstitial-diffusion limited",
        "SEI on cracks": "true",
        "SEI film resistance": "distributed", 
        "SEI porosity change": "true",
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "loss of active material": ("stress-driven","none"),
        "lithium plating": "partially reversible",},
    "Positive electrode LAM constant proportional term [s-1]": 1e-16,
    "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
    "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
    "Ratio of Li-ion concentration change in electrolyte consider solvent consumption":1.0,
    'EC initial concentration in electrolyte [mol.m-3]':4541.0,
    'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 
}

In [25]:
Scan_start = 1;    Scan_end = 10; 
Indexs =np.arange(Scan_start-1,Scan_end)
Para_dict_list = []

def load_combinations_from_csv(filename):
    dataframe = pd.read_csv(filename)
    parameter_names = dataframe.columns.tolist()
    combinations = dataframe.values.tolist()
    return parameter_names, combinations

# Usage example:
filename = BasicPath+Target+'Random_1000sets.csv'
parameter_names, combinations = load_combinations_from_csv(filename)

for index in Indexs:
    para_active = {}
    for parameter_name,para_value in zip(parameter_names,combinations[index] ):
        para_active[parameter_name] = para_value
    para_sum = {**para_active,**para_dict_Same}
    Para_dict_list.append(para_sum)

In [29]:
Para_dict_list[6]

{'Inner SEI lithium interstitial diffusivity [m2.s-1]': 1.32e-19,
 'Dead lithium decay constant [s-1]': 1.48e-06,
 'Lithium plating kinetic rate constant [m.s-1]': 7.7e-09,
 'Negative electrode LAM constant proportional term [s-1]': 1.95e-08,
 'Negative electrode cracking rate': 2.8e-20,
 'Initial electrolyte excessive amount ratio': 1.022553953,
 'EC partial molar volume [m3.mol-1]': 6.23e-05,
 'Contact resistance [Ohm]': 0.013886976,
 'Ageing temperature': 25,
 'Total ageing cycles': 6,
 'Ageing cycles between RPT': 1,
 'Update cycles for ageing': 1,
 'Cycles within RPT': 1,
 'RPT temperature': 25,
 'Mesh list': [5, 5, 5, 60, 20],
 'Para_Set': 'OKane2023',
 'Model option': {'contact resistance': 'true',
  'open-circuit potential': 'current sigmoid',
  'thermal': 'lumped',
  'SEI': 'interstitial-diffusion limited',
  'SEI on cracks': 'true',
  'SEI film resistance': 'distributed',
  'SEI porosity change': 'true',
  'particle mechanics': ('swelling and cracking', 'swelling only'),
  'l